# all text encoders

In [1]:
import pprint
pp = pprint.PrettyPrinter(indent=4, width=200, depth=None, stream=None, compact=False, sort_dicts=False)

In [2]:
import json

with open('../pdf_to_dictionary/ellentetek.json', encoding='utf-8') as f:
    d = json.load(f)

In [3]:
# make smaller test dict
d_keys_li=list(d.keys())
smaller_dict={}
for i in range(0,10):
    smaller_dict[d_keys_li[i]]=d[d_keys_li[i]]
# pp.pprint(smaller_dict)

In [4]:
# d - full test disctionary
# smaller_dict - 10 key dictionary
# DICT = smaller_dict
DICT = d

In [5]:
def get_avg_res_bert(tokenizer,model):
    from scipy.spatial import distance
    import numpy as np

    sum=0
    cnt=0
    avg=0

    for k,v in DICT.items():
        baseword=k
        encoded_input_base=tokenizer(baseword,return_tensors='pt')
        output_base = model(**encoded_input_base)
        for e in v[1]:
            synonym=e
            encoded_input_synonym=tokenizer(synonym,return_tensors='pt')
            output_synonym = model(**encoded_input_synonym)

            embedded_base_avg=np.average(output_base.last_hidden_state[0].detach().numpy(), axis=0)
            embedded_synonym_avg=np.average(output_synonym.last_hidden_state[0].detach().numpy(), axis=0)
            dist=distance.cosine(embedded_base_avg,embedded_synonym_avg)
            sum=sum+dist
            cnt=cnt+1
    # overall result on synonym-dictionary
        print(cnt, end="\r")
           
    avg=sum/cnt
    print(avg)
    return avg

In [6]:
def get_avg_res_other(tokenizer, model):
    from scipy.spatial import distance
    import numpy as np

    sum=0
    cnt=0
    avg=0

    for k,v in DICT.items():
        baseword=k
        encoded_input_base=tokenizer(baseword,return_tensors='pt')
        output_base = model(**encoded_input_base,output_hidden_states=True)
        output_base_np=output_base.hidden_states[-1][0].detach().numpy()
        for e in v[1]:
            synonym=e
            encoded_input_synonym=tokenizer(synonym,return_tensors='pt')
            output_synonym = model(**encoded_input_synonym,output_hidden_states=True)
            output_synonym_np=output_synonym.hidden_states[-1][0].detach().numpy()

            embedded_base_avg=np.average(output_base_np, axis=0)
            embedded_synonym_avg=np.average(output_synonym_np, axis=0)
            dist=distance.cosine(embedded_base_avg,embedded_synonym_avg)
            sum=sum+dist
            cnt=cnt+1
        print(cnt, end="\r")
    # overall result on synonym-dictionary       
    avg=sum/cnt
    print(avg)
    return avg

In [7]:
def get_avg_res_xlm100(tokenizer,model):
    from scipy.spatial import distance

    sum=0
    cnt=0
    avg=0

    language_id_hu = tokenizer.lang2id["hu"]

    for k,v in DICT.items():
        baseword=k
        base_input_ids = torch.tensor([tokenizer.encode(baseword)])
        base_lang =  torch.tensor([language_id_hu] * base_input_ids.shape[1])
        base_lang = base_lang.view(1, -1)
        output_base = model(base_input_ids, langs=base_lang)
        for e in v[1]:
            synonym=e
            syn_input_ids = torch.tensor([tokenizer.encode(synonym)])
            syn_lang =  torch.tensor([language_id_hu] * syn_input_ids.shape[1])
            syn_lang = syn_lang.view(1, -1)
            output_syn = model(syn_input_ids, langs=syn_lang)

            embedded_base=output_base[0][0][-1]
            embedded_syn=output_syn[0][0][-1]
            dist=distance.cosine(embedded_base.tolist(),embedded_syn.tolist())
            sum=sum+dist
            cnt=cnt+1
    # overall result on synonym-dictionary       
    avg=sum/cnt
    print(avg)
    return avg

In [7]:
# dic_result = {
#     "bert-base-multilingual-uncased" : {},
#     "bert-base-multilingual-cased" : {},
#     "NYTK/text-generation-news-gpt2-small-hungarian" : {},
#     "NYTK/text-generation-poem-petofi-gpt2-small-hungarian" : {},
#     "SZTAKI-HLT/hubert-base-cc" : {},
#     "sberbank-ai/mGPT" : {},
#     "NYTK/sentiment-hts2-xlm-roberta-hungarian" : {},
#     "NYTK/sentiment-hts5-xlm-roberta-hungarian" : {},
#     "NYTK/sentiment-hts5-hubert-hungarian" : {},
#     "NYTK/sentiment-hts2-hubert-hungarian" : {},
#     "roberta-base" : {},
#     "xlm-roberta-base" : {},
#     "xlm-roberta-large" : {},
#     "xlm-mlm-100-1280" : {},
#     "facebook/xlm-roberta-xl" : {}
# }
# with open("results.json", "w+") as resultf:
#     json.dump(dic_result, resultf)

In [8]:
with open("results.json", "r") as resultf:
    dic_result = json.load(resultf)
dic_result

{'bert-base-multilingual-uncased': {'syn_hu': 0.27792526276512247},
 'bert-base-multilingual-cased': {'syn_hu': 0.4367072685744003},
 'NYTK/text-generation-news-gpt2-small-hungarian': {'syn_hu': 0.42827802149637384},
 'NYTK/text-generation-poem-petofi-gpt2-small-hungarian': {'syn_hu': 0.3693537550153803},
 'SZTAKI-HLT/hubert-base-cc': {'syn_hu': 0.05893374912235821},
 'sberbank-ai/mGPT': {'syn_hu': 0.1330561433268905},
 'NYTK/sentiment-hts2-xlm-roberta-hungarian': {'syn_hu': 0.6582233949544475},
 'NYTK/sentiment-hts5-xlm-roberta-hungarian': {'syn_hu': 0.1807633529347575},
 'NYTK/sentiment-hts5-hubert-hungarian': {'syn_hu': 0.1294809751615292},
 'NYTK/sentiment-hts2-hubert-hungarian': {'syn_hu': 0.16887050825909086},
 'roberta-base': {},
 'xlm-roberta-base': {'syn_hu': 0.006440796056190049},
 'xlm-roberta-large': {'syn_hu': 0.005448855006477714},
 'xlm-mlm-100-1280': {'syn_hu': 0.423519610398453},
 'facebook/xlm-roberta-xl': {'syn_hu': 0.05564785176719958}}

In [ ]:
# distilbert-base-multilingual-cased
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-multilingual-cased")
model = AutoModelForMaskedLM.from_pretrained("distilbert-base-multilingual-cased")

dic_result["distilbert-base-multilingual-cased"]["ant_hu"] = get_avg_res_other(tokenizer,model)

with open("results.json", "w+") as resultf: json.dump(dic_result, resultf)

In [9]:
# bert-base-multilingual-uncased
# Run time: 21m
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-uncased")
model = BertModel.from_pretrained("bert-base-multilingual-uncased")

dic_result["bert-base-multilingual-uncased"]["ant_hu"] = get_avg_res_bert(tokenizer,model)

with open("results.json", "w+") as resultf: json.dump(dic_result, resultf)

c:\Users\L0021\Anaconda3\envs\test\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be

0.2827126649011932


In [10]:
# bert-base-multilingual-cased
# Run time: 19m
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
model = BertModel.from_pretrained("bert-base-multilingual-cased")

dic_result["bert-base-multilingual-cased"]["ant_hu"] = get_avg_res_bert(tokenizer,model)

with open("results.json", "w+") as resultf: json.dump(dic_result, resultf)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


0.4390978391437541


In [11]:
# hubert-base-cc
# Run time: 18m
from transformers import AutoModel, AutoTokenizer

model = AutoModel.from_pretrained("SZTAKI-HLT/hubert-base-cc")
tokenizer = AutoTokenizer.from_pretrained("SZTAKI-HLT/hubert-base-cc")

dic_result["SZTAKI-HLT/hubert-base-cc"]["ant_hu"] = get_avg_res_bert(tokenizer,model)

with open("results.json", "w+") as resultf: json.dump(dic_result, resultf)

Some weights of the model checkpoint at SZTAKI-HLT/hubert-base-cc were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


0.06082000361830001


In [12]:
# text-generation-news-gpt2-small-hungarian
# Run time: 21m
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("NYTK/text-generation-news-gpt2-small-hungarian")
model = AutoModelForCausalLM.from_pretrained("NYTK/text-generation-news-gpt2-small-hungarian")

dic_result["NYTK/text-generation-news-gpt2-small-hungarian"]["ant_hu"] = get_avg_res_other(tokenizer,model)

with open("results.json", "w+") as resultf: json.dump(dic_result, resultf)

0.458458951674369


In [13]:
# text-generation-poem-petofi-gpt2-small-hungarian
# Run time: 21m
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("NYTK/text-generation-poem-petofi-gpt2-small-hungarian")
model = AutoModelForCausalLM.from_pretrained("NYTK/text-generation-poem-petofi-gpt2-small-hungarian")

dic_result["NYTK/text-generation-poem-petofi-gpt2-small-hungarian"]["ant_hu"] = get_avg_res_other(tokenizer,model)

with open("results.json", "w+") as resultf: json.dump(dic_result, resultf)

0.3993917938213175


In [14]:
# mGPT
# Run time: 234m
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/mGPT")
model = AutoModelForCausalLM.from_pretrained("sberbank-ai/mGPT")

dic_result["sberbank-ai/mGPT"]["ant_hu"] = get_avg_res_other(tokenizer,model)

with open("results.json", "w+") as resultf: json.dump(dic_result, resultf)

0.1406258561842439


In [15]:
# sentiment-hts2-xlm-roberta-hungarian
# Run time: 11m
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("NYTK/sentiment-hts2-xlm-roberta-hungarian")
model = AutoModelForSequenceClassification.from_pretrained("NYTK/sentiment-hts2-xlm-roberta-hungarian")

dic_result["NYTK/sentiment-hts2-xlm-roberta-hungarian"]["ant_hu"] = get_avg_res_other(tokenizer,model)

with open("results.json", "w+") as resultf: json.dump(dic_result, resultf)

0.7164178789762122


In [16]:
# sentiment-hts5-xlm-roberta-hungarian
# Run time: 18m
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("NYTK/sentiment-hts5-xlm-roberta-hungarian")
model = AutoModelForSequenceClassification.from_pretrained("NYTK/sentiment-hts5-xlm-roberta-hungarian")

dic_result["NYTK/sentiment-hts5-xlm-roberta-hungarian"]["ant_hu"] = get_avg_res_other(tokenizer,model)

with open("results.json", "w+") as resultf: json.dump(dic_result, resultf)

0.23405399693724285


In [17]:
# sentiment-hts5-hubert-hungarian
# Run time: 18m
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("NYTK/sentiment-hts5-hubert-hungarian")
model = AutoModelForSequenceClassification.from_pretrained("NYTK/sentiment-hts5-hubert-hungarian")

dic_result["NYTK/sentiment-hts5-hubert-hungarian"]["ant_hu"] = get_avg_res_other(tokenizer,model)

with open("results.json", "w+") as resultf: json.dump(dic_result, resultf)

0.17378702840269716


In [18]:
# sentiment-hts2-hubert-hungarian
# Run time: 17m
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("NYTK/sentiment-hts2-hubert-hungarian")
model = AutoModelForSequenceClassification.from_pretrained("NYTK/sentiment-hts2-hubert-hungarian")

dic_result["NYTK/sentiment-hts2-hubert-hungarian"]["ant_hu"] = get_avg_res_other(tokenizer,model)

with open("results.json", "w+") as resultf: json.dump(dic_result, resultf)

0.2911613891161046


# This is english only
# roberta-base
# Run time: 22m
from transformers import RobertaTokenizer, RobertaModel
import torch

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
model = RobertaModel.from_pretrained("roberta-base")

get_avg_res(tokenizer,model)

In [19]:
# xlm-roberta-base
# Run time: 43m
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
model = AutoModelForMaskedLM.from_pretrained("xlm-roberta-base")

dic_result["xlm-roberta-base"]["ant_hu"] = get_avg_res_other(tokenizer,model)

with open("results.json", "w+") as resultf: json.dump(dic_result, resultf)

0.006509641419028101


In [20]:
# xlm-roberta-large
# Run time: 82m
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-large')
model = AutoModelForMaskedLM.from_pretrained("xlm-roberta-large")

dic_result["xlm-roberta-large"]["ant_hu"] = get_avg_res_other(tokenizer,model)

with open("results.json", "w+") as resultf: json.dump(dic_result, resultf)

0.0055712449077952066


In [21]:
# xlm-mlm-100-1280
# Run time: 109m
from transformers import XLMTokenizer, XLMWithLMHeadModel
import torch

tokenizer = XLMTokenizer.from_pretrained("xlm-mlm-100-1280")
model = XLMWithLMHeadModel.from_pretrained("xlm-mlm-100-1280")

dic_result["xlm-mlm-100-1280"]["ant_hu"] = get_avg_res_other(tokenizer,model)

with open("results.json", "w+") as resultf: json.dump(dic_result, resultf)

Some weights of XLMWithLMHeadModel were not initialized from the model checkpoint at xlm-mlm-100-1280 and are newly initialized: ['transformer.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


0.43119427910552915


In [22]:
# xlm-roberta-xl
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("facebook/xlm-roberta-xl")
model = AutoModelForMaskedLM.from_pretrained("facebook/xlm-roberta-xl")

dic_result["facebook/xlm-roberta-xl"]["ant_hu"] = get_avg_res_other(tokenizer,model)

with open("results.json", "w+") as resultf: json.dump(dic_result, resultf)

0.05678474388908368
